# Object Detection

* [SSD-Mobilenet]
* [SSD-Resnet50]


[Docker Containers of the network](https://github.com/ctuning/ck-object-detection/tree/master/docker)


* [FPN-SSD]
* [RCNN inception ResNet v2]
* [RCNN nas lowproposal]
* [RCNN ResNet101 lowproposal]
* [RCNN ResNet50 lowproposal]
* [RCNN nas]

## Table of Contents

1. [Overview](#overview)
1. [Platform](#platform)
1. [Experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]
1. [Experiments on Hikey](#experiments_hikey)
   1. [TensorFlow](#experiments_tensorflow_hikey)
1. [Experiments on Firefly](#experiments_firefly)
   1. [TensorFlow](#experiments_tensorflow_firefly)

<a id="overview"></a>
## Overview

This Jupyter Notebook studies performance (execution time) vs accuracy (mAP and Recall) of different Object Detection networks, on different size objects (large, medium and small).
Moreover the experiments are performed on different architectures, in particular CPU and GPU, to evaluate the benefit of GPU acceleration.

<a id="platform"></a>
## Platform

<a id="CPU info"></a>
### CPU

  - Model:
    - Intel Xeon
  - Version:
    - E5-2650 v3;  
  - Frequency:
    - 2.30GHz;
  - Number of Cores (physical):
    - 10
  - HyperThreading
    - Yes


  - RAM:
    - 32 GB;

  - BSP:
    - Ubuntu 16.04 LTS Linux


<a id="GPU info"></a>
### GPU

  - Model:
    - Nvidia GeForce GTX 1080
  - Frequency:
    - 1.6GHz;
  - CUDA Version:
    - 10.2
  - Driver Version:
    - 430.14
  - MEMORY:
    - 8 GB;


<a id="data"></a>
## Get the experimental data

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

If data are not present, download and add the repository to ck using:
```
!wget https://www.dropbox.com/s/532l5yb1qvxp8q6/my_experiments_0.zip?dl=0

!mv my_experiments_0.zip\?dl\=0 my_experiments_0.zip
!ck add repo:my-repo-with-experiments --zip=my_experiments_0.zip
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

!wget https://www.dropbox.com/s/532l5yb1qvxp8q6/my_experiments_0.zip?dl=0

!mv my_experiments_0.zip\?dl\=0 my_experiments_0.zip
!ck add repo:my_repo2 --zip=my_experiments_0.zip

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)


    
repo_uoa = 'object-detection-accuracy'
!ck list $repo_uoa:experiment:* | sort
        
print ("*****************************************")


repo_perf_uoa = 'object-detection-performances'
!ck list $repo_perf_uoa:experiment:* | sort

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='', accuracy=True,
                             module_uoa='experiment', _library=None, _platform=None):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    from pprint import pprint
    #pprint (r)
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']

    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
  #      pprint(r)
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        # Mapping of expected library tags to reader-friendly names.
        tag_to_name = {
            # ArmCL tags on HiKey.
            '17.12-48bc34ea'    : 'armcl-17.12',
            '18.01-f45d5a9b'    : 'armcl-18.01',
            '18.03-e40997bb'    : 'armcl-18.03',
            'request-d8f69c13'  : 'armcl-dv/dt', # armcl-18.03+
            '18.05-b3a371bc'    : 'armcl-18.05',
            # ArmCL tags on Firefly.
            '17.12-48bc34e'     : 'armcl-17.12',
            '18.01-f45d5a9'     : 'armcl-18.01',
            '18.03-e40997b'     : 'armcl-18.03',
            '18.05-b3a371b'     : 'armcl-18.05',
            # TensorFlow tags.
            'tensorflow-1.7'    : 'tensorflow-1.7',
            'tensorflow-1.8'    : 'tensorflow-1.8',
        }
            
        # Library.
        library_tags = [ tag for tag in r['dict']['tags'] if tag in tag_to_name.keys() ]
        #if len(library_tags)==1:
        #    library = tag_to_name[library_tags[0]]
        #else:
        #    print('[Warning] Bad library tags. Skipping experiment with tags:')
        #    print(r['dict']['tags'])
        #    continue
        #if _library and _library!=library: continue
        # For each point.   
        
        pipeline_file_path = os.path.join(r['path'], 'pipeline.json')
        with open(pipeline_file_path) as pipeline_file:
            pipeline_data_raw = json.load(pipeline_file)
        weights_env = pipeline_data_raw['dependencies']['weights']['dict']['env']
 #       pprint(weights_env)
        
        tags = r['dict']['tags']
        if tags[1] == 'ssd-mobilenet-non-quantized':
            continue
        print (tags)
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']

#            pprint(characteristics_list)
#            print ("****************************")
#            print (point_data_raw.keys())
#            print ("****************************")
            
            pprint (point_data_raw['choices']['env'])
            num_repetitions = len(characteristics_list)
            platform = point_data_raw['features']['platform']['platform']['model']
            #if _platform and _platform!=platform: continue
            img_width = np.int64(point_data_raw['choices']['env'].get('CK_ENV_IMAGE_WIDTH',-1))
            img_height = np.int64(point_data_raw['choices']['env'].get('CK_ENV_IMAGE_HEIGHT',-1))
            if np.int64(point_data_raw['choices']['env'].get('CK_ENABLE_BATCH',-1))==1:
                batch_en = True 
                batch_size = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))
                batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))
            else :
                batch_size = 1
                batch_en = False 
                batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))*np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))

            characteristics = characteristics_list[0]
            #convolution_method = convolution_method_to_name[np.int64(point_data_raw['choices']['env'].get('CK_CONVOLUTION_METHOD_HINT',1))]
            #if library.startswith('tensorflow-'):
            #    multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_MULTIPLIER',-1))
            #    resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_TENSORFLOW_MODEL_MOBILENET_RESOLUTION',-1))
            #else:
            #    multiplier = np.float64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_WIDTH_MULTIPLIER',-1))
            #    resolution = np.int64(point_data_raw['choices']['env'].get('CK_ENV_MOBILENET_RESOLUTION',-1))
            #model = 'v1-%.2f-%d' % (multiplier, resolution)
            if accuracy:
                data = [
                    {
                        # features
                        #'platform': platform,
                        # choices
                        #'weights_env': weights_env['CK_ENV_TENSORFLOW_MODEL_FROZEN_GRAPH'],
                        'model': tags[1],# if len (tags) == 2 else tags[0],
                        'tf_version':'cuda',#tags[0],
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'img_height': img_height,
                        'img_width':tags[2],
                        'num_reps':1,
                        
                        # runtime characteristics
                        'mAP': characteristics['run'].get('mAP', 0)*100,
                        'Recall': characteristics['run'].get('recall', 0)*100,
                        'mAP_large': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (large)', 0)*100,
                        'mAP_medium': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (medium)', 0)*100,
                        'mAP_small': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (small)', 0)*100,
#                         # recompute accuracy from frame_predictions (was incorrectly recorded in early experiments)
#                         'accuracy_top1_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top1']=='yes'
#                         ]) / np.float64(batch_count),
#                         'accuracy_top5_': len([
#                             prediction for prediction in characteristics['run'].get('frame_predictions', [])
#                             if prediction['accuracy_top5']=='yes'
#                         ]) / np.float64(batch_count)
                    }
#                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
                print(data[0]['model'])
            else: # performance
                tf_version = 'default'
                trt = point_data_raw['choices']['env'].get('CK_ENABLE_TENSORRT',0) 
                trt_dyn = point_data_raw['choices']['env'].get('CK_TENSORRT_DYNAMIC',0)
                
                print (point_data_raw['choices']['env'].get('CK_ENABLE_TENSORRT',0),trt)        
                print (point_data_raw['choices']['env'].get('CK_TENSORRT_DYNAMIC',0),trt_dyn)        
                if trt_dyn == '1':
                    tf_version = 'tensorRT_dynamic'
                elif trt == '1':
                    tf_version = 'tensorRT'
                elif tags[0] == 'tensorrt':
                    tf_version = 'cuda_sources'
                else:
                    tf_version = tags[0]
                data = [
                    {
                        # features
                        #'platform': platform,
                        # choices
                        #'weights_env': weights_env['CK_ENV_TENSORFLOW_MODEL_FROZEN_GRAPH'], 
                        #'model': weights_env['CK_ENV_TENSORFLOW_MODEL_MODEL_NAME'],#tags[0],

                        
                        'model': tags[1],# if len (tags) == 2 else tags[0],
                        'tf_version':tf_version,
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'batch_en': batch_en,
                        'img_height': img_height,
                        'img_width':img_width,
                        'num_reps' : num_repetitions,
                        # statistical repetition
                        'repetition_id': repetition_id,
                        # runtime characteristics
                        'avg_fps': characteristics['run'].get('avg_fps', 'n/a')*batch_size,
                        'avg_time_ms': characteristics['run']['avg_time_ms']/batch_size,
                        'graph_load_time_ms': characteristics['run']['graph_load_time_s']*1e+3,
                        'images_load_time_avg_ms': characteristics['run']['images_load_time_avg_s']*1e+3,
                    }
                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
                print(data[0]['tf_version'])
            index = [
                'model', 'tf_version', 'batch_size', 'batch_count','batch_en','img_height','img_width','num_reps'
                #, 'mAP', 'Recall', 'mAP_large', 'mAP_medium', 'mAP_small'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success?'])
    return result
!ck recache repo
#dfs = get_experimental_results(repo_uoa)

dfs_per = get_experimental_results(repo_perf_uoa,accuracy=False)

### Plot experimental data

In [ ]:
display_in_full(dfs)

#print ("**********************")


display_in_full(dfs_per)

#### Plot accuracy (bar plot)

In [ ]:
def plot_accuracy(df_raw, groupby_level='img_height', performance_metric=['mAP','mAP_large','mAP_medium','mAP_small'], title=None, figsize=None, rot=90):
    #from pprint import pprint
    #pprint(df_raw[performance_metric].values)
    #pprint(df_raw.index.values)
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            #tuples=[ (m,l,s,c) for (m,l,_,_,s,c) in df_raw.index.values ],
            #names=[ 'tf_version', 'model', 'img_height', 'img_width' ]
            tuples=[ (l,s,c) for (l,_,_,_,s,c,_) in df_raw.index.values ],
            names=[ 'model', 'img_height', 'img_width' ]
        )
    )
    #groupby decide quanti plot fare.
    #unstack come separare le colonne (?)
    unstack_level = 'img_height'
    colormap = cm.autumn
    xlabel='(model)'
    #df_bar.columns.names = ['time']
    #if groupby_level=='convolution_method':
    #    unstack_level = 'library'
    #    xlabel='(Model [channel multiplier - input resolution], Convolution Method)'
    #    colormap = cm.autumn
    #elif groupby_level=='library':
    #    unstack_level = 'convolution_method'
    #    xlabel='(Library, Model [channel multiplier - input resolution])'
    #    colormap = cm.summer
    # Set default style.
    
    ylabel='mAP %'
    resize_dim = ['no_resize', 'model_resize','400*600', '480*600','600*600','900*900']
    if not title: title = 'prova' 
    if not figsize: figsize = [default_figwidth, 8]
    #pprint (df_bar.index.names[:-1])
    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean()#.unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std()#.unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    for count, ax in enumerate(axes):
        # Title.
        ax.set_title(resize_dim[count])
        # X label.
        ax.set_xlabel(xlabel)
        # Y axis.
        ax.set_ylabel(ylabel)

plot_accuracy(dfs , rot=90)#,performance_metric=['mAP'])#,'mAP_large','mAP_medium','mAP_small'])


#### Plot performance (bar plot)

In [ ]:
def plot_performance(df_raw, groupby_level='tf_version', performance_metric=['avg_fps','avg_time_ms','graph_load_time_ms','images_load_time_avg_ms'], title=None, figsize=None, rot=90):
    from pprint import pprint
    #pprint(df_raw[performance_metric].values)
    #pprint(df_raw.index.values)
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
 #           tuples=[ (m,t,s,c) for (m,t,s,c,_,_,_) in df_raw.index.values ],
 #           names=[ 'model','tf_version', 'batch_size', 'batch_count' ]
            tuples=[ (l,t,c,s) for (l,t,_,_,s,c,_) in df_raw.index.values ],
            names=[ 'model','tf_version', 'img_width', 'img_height' ]            
        )
    )
    #groupby decide quanti plot fare.
    #unstack come separare le colonne (?)
    unstack_level = 'img_width'
    colormap = cm.autumn
    xlabel='(model)'
    #df_bar.columns.names = ['time']
    #if groupby_level=='convolution_method':
    #    unstack_level = 'library'
    #    xlabel='(Model [channel multiplier - input resolution], Convolution Method)'
    #    colormap = cm.autumn
    #elif groupby_level=='library':
    #    unstack_level = 'convolution_method'
    #    xlabel='(Library, Model [channel multiplier - input resolution])'
    #    colormap = cm.summer
    # Set default style.
    ylabel=''
    if not title: title = df_bar.index.names[0]
    #print ('*********************************************')

    #pprint(df_bar)    
    #print ('*********************************************')

    #pprint(df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('tf_version'))            
    if not figsize: figsize = [default_figwidth, 8]

    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean()#.unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std()#.unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    #pprint (axes.keys().get_values().item(0))
    
    for num, ax in enumerate(axes):
        # Title.
        ax.set_title(axes.keys().get_values().item(num))
        # X label.
        ax.set_xlabel(xlabel)
        # Y axis.
        ax.set_ylabel(ylabel)
plot_performance(dfs_per, performance_metric=['avg_fps'])
#plot_performance(dfs_per, performance_metric=['avg_time_ms'])
#plot_performance(dfs_per, performance_metric=['graph_load_time_ms','images_load_time_avg_ms'])

In [ ]:
def merge_performance_accuracy(df_performance, df_accuracy, 
                               reference_platform=None, reference_lib=None, 
                               performance_metric='avg_fps', accuracy_metric='mAP'):
    df = df_performance[[performance_metric]]
    #print (df)
    accuracy_list = []
    for index, row in df.iterrows():
        (model,tf_version, batch_size, batch_count,batch_en, img_height,img_width,num_reps) = index
        print (model,tf_version, batch_size, batch_count,batch_en, img_height,img_width,num_reps)
        #img_size = 'no-resize' if batch_size == 1 else 'model-resize'
        #print (img_size)
        #print ((model,tf_version, batch_size, batch_count, img_height,img_width,num_reps))
        accuracy = df_accuracy.loc[(model,'cuda', 1, 5000, -1 ,'no-resize',1)][accuracy_metric]
        accuracy_list.append(accuracy)

    print ("***************")

    
    df = df.assign(mAP=accuracy_list) # FIXME: assign to the value of accuracy_metric
    print (df[['avg_fps','mAP']])
    return df
dfs_per_acc = merge_performance_accuracy(dfs_per, dfs,performance_metric="avg_fps", accuracy_metric="mAP" )

In [ ]:
def plot(df_performance_accuracy,  
         performance_metric='avg_fps', accuracy_metric='mAP',
         xmin=0.0, xmax=80, xstep=5.0, ymin=20, ymax=65, ystep=5,
         title=None, save_fig=False, save_fig_name='mobilenets-default'):
    fig = plt.figure(figsize=(8,4), dpi=default_figdpi)
    ax = fig.gca()
    
    model_to_color = { 
        'fpn-ssd'                                : 'red',
        'inception-lowproposal-rcnn-resnetv2'    : 'yellow',
        'lowproposal-nas-rcnn'                   : 'orange',
        'lowproposal-rcnn-resnet101'             : 'green',
        'rcnn-inceptionv2'                       : 'purple',
        'rcnn-nas-non-lowproposal'               : 'cyan',
        'ssd-inceptionv2'                        : 'blue',
        'ssd-mobilenet-non-quantized'            : 'gray',
        'ssd-mobilenet-quantized'                : 'indigo',
        'ssd-resnet50'                           : 'saddlebrown',
        'ssdlite'                                : 'teal',
        'lowproposal-rcnn-resnet50'              : 'darkgoldenrod',
        'yolo'                                   : 'brown'
    }
    model_to_real_name = { 
        'fpn-ssd'                                : 'ssd_mobilenet_v1_fpn_coco',
        'inception-lowproposal-rcnn-resnetv2'    : 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco',
        'lowproposal-nas-rcnn'                   : 'faster_rcnn_nas_lowproposals_coco',
        'lowproposal-rcnn-resnet101'             : 'faster_rcnn_resnet101_lowproposals_coco',
        'rcnn-inceptionv2'                       : 'faster_rcnn_inception_resnet_v2_atrous_coco',
        'rcnn-nas-non-lowproposal'               : 'faster_rcnn_nas',
        'ssd-inceptionv2'                        : 'ssd_inception_v2_coco',
        'ssd-mobilenet-non-quantized'            : 'ssd_mobilenet_v1_coco',
        'ssd-mobilenet-quantized'                : 'ssd_mobilenet_v1_quantized_coco',
        'ssd-resnet50'                           : 'ssd_resnet_50_fpn_coco',
        'ssdlite'                                : 'ssdlite_mobilenet_v2_coco',
        'lowproposal-rcnn-resnet50'              : 'faster_rcnn_resnet50_lowproposals_coco',
        'yolo'                                   : 'yolo v3'
    }
    tf_to_marker = {
        'tensorRT'             : 'x',
        'tensorRT_dynamic'     : '*',
        'cuda_sources'         : 'D',
        'tf-src-cuda'          : 'D',
        'tf-src-cpu'           : '<',
        'tf-prebuild-cpu'      : 'o'
    }
    resize_to_marker = {
        'no-resize'             : 'x',
        'model-resize'          : '*'

    }
    
    print (df_performance_accuracy)
    df = df_performance_accuracy
    
    for index, row in df.iterrows():
        (model,tf_version, batch_size, batch_count,batch_en, img_height,img_width,num_reps) = index
        performance = row[performance_metric]
        accuracy = row[accuracy_metric]
        
        # Mark Pareto-optimal points.
        is_on_pareto = True
        for index1, row1 in df.iterrows():
            is_no_slower = row1[performance_metric] >= row[performance_metric]
            is_no_less_accurate = row1[accuracy_metric] >= row[accuracy_metric]
            is_faster = row1[performance_metric] > row[performance_metric]
            is_more_accurate = row1[accuracy_metric] > row[accuracy_metric]
            if ((is_faster and is_no_less_accurate) or (is_more_accurate and is_no_slower)):
                print ("tested point has:",performance,accuracy)
                print ("counterexample has: " ,row1[performance_metric],row1[accuracy_metric])
                is_on_pareto = False
                break
        if is_on_pareto:
            print ("point:")
            print (performance,accuracy,tf_version,model)
            print ("is pareto optimal")
#        # GEMM-based convolution should be exactly the same in '18.03' and 'dv/dt', so plot
#        # the minimum execution time of '18.03' and 'dv/dt' as '18.03'.
#        if 'armcl-dv/dt' in libs and convolution_method=='gemm' and (lib=='armcl-dv/dt' or lib=='armcl-18.03'):
#            performance_dv_dt = df.loc[('armcl-dv/dt', model, multiplier, resolution, batch_size, convolution_method)][performance_metric]
#            performance_18_03 = df.loc[('armcl-18.03', model, multiplier, resolution, batch_size, convolution_method)][performance_metric]
#            if lib=='armcl-18.03':
#                if (performance_dv_dt < performance_18_03):
#                    continue
#            if lib=='armcl-dv/dt':
#                if (performance_dv_dt < performance_18_03):
#                    lib = 'armcl-18.03' # change color
#                else:
#                    continue
        
        # Select size, color and marker.
        size = 5 #resolution / 16
        color = model_to_color[model]
        marker = tf_to_marker[tf_version]#resize_to_marker[img_width]#

        # Plot.
        #print (performance,accuracy,tf_version,model)
        #print (performance,accuracy,tf_to_marker[tf_version],model_to_color[model])
        ax.plot(performance, accuracy, marker, markerfacecolor=color, markersize=size,markeredgecolor=color)

        # Mark Pareto-optimal points with scaled black pluses.
        if is_on_pareto:
            ax.plot(performance, accuracy, 'k+', markersize=0.5*size)

    # Title.
    if not title: "bla"#title = '%s (GPU: %s @ %s)' % (id_to_name[platform_id], id_to_gpu[platform_id], id_to_gpu_mhz[platform_id])
    ax.set_title(title)
    # X axis.
    xlabel='fps'#'Image recognition time (ms)' if performance_metric=='time_avg_ms' else ''
    ax.set_xlabel(xlabel)
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax, xstep))
    for xtick in ax.xaxis.get_major_ticks(): xtick.label.set_fontsize(12)
    # Y axis.
    ylabel='mAP'#'Image recognition accuracy (top %s)' % accuracy_metric[-1]
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(ymin, ymax, ystep))
    for ytick in ax.yaxis.get_major_ticks(): ytick.label.set_fontsize(12)
    # Legend.
    handles = [ 
        mp.patches.Patch(color=color, label=model_to_real_name[model])
        for (model, color) in sorted(model_to_color.items())
        
        #if label in models
    ]
    print (handles)

    import matplotlib.lines as mlines

    mark1 = mlines.Line2D([], [], color='black', marker='x', linestyle='None',
                          markersize=5, label='tensorRT')
    mark2 = mlines.Line2D([], [], color='black', marker='*', linestyle='None',
                          markersize=5, label='tensorRT_dynamic')
    mark3 = mlines.Line2D([], [], color='black', marker='D', linestyle='None',
                          markersize=5, label='cuda_sources')
    mark4 = mlines.Line2D([], [], color='black', marker='<', linestyle='None',
                          markersize=5, label=' tf-src-cpu')
    mark5 = mlines.Line2D([], [], color='black', marker='o', linestyle='None',
                          markersize=5, label=' tf-prebuild-cpu')
    

#    mark1 = mlines.Line2D([], [], color='black', marker='x', linestyle='None',
#                          markersize=5, label='no resize')
#    mark2 = mlines.Line2D([], [], color='black', marker='*', linestyle='None',
#                          markersize=5, label='model resize')
    
    handles2 = [     mark1,mark2,mark3,mark4,mark5   ]
    #handles2 = [     mark1,mark2    ]
    handles+=handles2
    plt.legend(title='Library', handles=handles[::-1], loc='upper right', prop={'size': 5})
    # Show with grid on.
    plt.grid(True)
    plt.show()
    # Save figure.
    if save_fig:
        save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name, save_fig_ext))
        plt.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')
plot (dfs_per_acc)

Resizing layer in the network:

there are two different types of resizing: fixed and keep aspect ratio.

The first one takes as input the image and returns an image of fixed dimensions, changing from network to network.
Using this layer are the following networks:

       - 'ssd_mobilenet_v1_fpn_coco', 640*640
       - 'faster_rcnn_nas_lowproposals_coco', 1200*1200
       - 'faster_rcnn_nas', 1200*1200
       - 'ssd_inception_v2_coco', 300*300
       - 'ssd_mobilenet_v1_coco', 300*300
       - 'ssd_mobilenet_v1_quantized_coco', 300*300
       - 'ssd_resnet_50_fpn_coco', 640*640
       - 'ssdlite_mobilenet_v2_coco', 300*300
       - 'yolo v3', 416*416
       
The second one behaviour is actually not completely clear, and have a minimum and maximum dimension of the output images. network using this layer are:

       - 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco', min: 600  max: 1024
       - 'faster_rcnn_inception_resnet_v2_atrous_coco', min: 600  max: 1024
       - 'faster_rcnn_nas', min: 600  max: 1024
       - 'faster_rcnn_resnet50_lowproposals_coco', min: 600  max: 1024
       
From the analysis on the performance-accuracy benchmarks, it seems that the models using fixed resizing performs better than the one with the keep aspect ratio.

yolo is actually in the middle ground: its preprocessing is doing a fixed resize to 416 * 416, however the resize is done keeping the aspect ratio and padding, outside the network. the network doesn't perform any resizing, but takes 416 * 416 images as input.
        